In [1]:
# 1. download HTML files to "html" directory
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

# kill previous chrome instance if still around (to conserve memory)
os.system("pkill -f -9 chromium")

options = Options()
options.headless = True
driver = webdriver.Chrome(options=options, executable_path="chromium.chromedriver")

# class="kn-responsive kn-table-table is-bordered is-striped knTable--borders knTable--spacing-medium knTable--striped"

def wait_for_element(elem_id):
    max_seconds = 10
    for i in range(max_seconds * 10):
        time.sleep(0.5)
        try:
            elem = driver.find_element_by_id(elem_id)
            break
        except NoSuchElementException:
            print("not ready yet")
    return "Found!"

if not os.path.exists("html"):
    os.mkdir("html")
    for i in range(1, 95):
        url = "https://madison.com/uw-salary-database/html_e37ff025-9a87-5a31-91ea-b2eb50aba4cb.html#2018-uw-salaries/?view_20_page="+str(i)+"&view_20_filters=%5B%7B%22field%22%3A%22field_106%22%2C%22operator%22%3A%22is%22%2C%22value%22%3A%22UW%20Madison%22%7D%5D&view_20_per_page=1000"
        print(url)
        driver.get(url)
        time.sleep(10)
        #wait_for_element("view_20")
        with open(os.path.join("html", str(i)+".html"), "w") as f:
            f.write(driver.page_source)

In [13]:
# 2. HTML to DataFrame
import pandas as pd
from bs4 import BeautifulSoup

def html_to_df(path):
    with open(path) as f:
        html = f.read()
    page = BeautifulSoup(html)

    # find the table
    table = page.find("table")
    rows = []    

    # find the rows
    for tr in table.find_all("tr"):
        cells = []
        for td in tr.find_all("td"):
                txt = td.find("span").get_text().strip()
                if td:
                    cells.append(txt)
        rows.append(cells)
    return pd.DataFrame(rows[1:])

dfs = []
for i in range(1, 95):
    df = html_to_df(os.path.join("html", str(i)+".html"))
    dfs.append(df)
    
df = pd.concat(dfs).reset_index()
df.head()

,index,0,1,2,3,4,5,6,7
0,0,2019,,Outreach Specialist,L&S/Administration/Admin,UW Madison,03/18/2019,"$16,538.99",view
1,1,2019,,Professor (CHS),Smph\Pediatrics\Child Pro,UW Madison,07/01/2006,"$76,594.00",view
2,2,2021,Shaun Aakre,Lecturer,Engr/Engr Prof Development,UW Madison,09/06/2015,"$15,679.64",view
3,3,2021,Lisa Aarli,Sr Advisor,L&S/Journ & Mass Com/Jn & Mass,UW Madison,01/01/2007,"$75,249.16",view
4,4,2020,Lisa Aarli,Sr Advisor,L&S/Journ & Mass Com/Jn & Mass,UW Madison,01/01/2007,"$70,888.50",view


In [21]:
# Clean up DataFrame
# df = df.drop(columns=['index', 4, 7])
# df = df.rename({0: 'Fiscal year', 1: 'Name', 2: 'Title', 3: 'Dept Description', 5: 'Start Date', 6: 'Total Pay'}, axis='columns')

In [22]:
# Save DataFrame to .csv
df.to_csv("salaries.csv", encoding='utf-8', index=False)